#### Stochastic programming with PySp

#### Installation

Step 1. Download source from https://github.com/Pyomo/pysp

Step 2. Browse to the dowloaded folder from your terminal (eg. Mac terminal, Windows CMD, Anaconda prompt)

Step 3. Run the command "python setup.py install"

#### Example : Newsvendor problem with 3 scenarios

* Solution with Pyomo and Indexing

In [6]:
from pyomo.environ import *

# Initialize problem data
model = ConcreteModel()
model.c = Param( initialize=0.6 )
model.r = Param( initialize=1.5 )

# define scenario dependent parameters
model.scenarios = range(3)
model.d = Param(model.scenarios, initialize=[15, 45, 90])
model.probability = Param(model.scenarios, default=1/3)

# define variables
model.x = Var(within=NonNegativeReals)
model.y = Var(model.scenarios, within=NonNegativeReals)

# objective
def obj_rule(model):
    return -model.c*model.x + sum(model.probability[s]*model.r*model.y[s] for s in model.scenarios)
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
def con1(model, s):
    return model.y[s] <= model.x
model.constr1 = Constraint(model.scenarios, rule=con1)

def con2(model, s):
    return model.y[s] <= model.d[s]
model.constr2 = Constraint(model.scenarios, rule=con2)

solver = SolverFactory("cbc")
solver.solve(model)

print("Objective : ", model.obj())

Objective :  25.5


* Solution with Pyomo and PySP

In [1]:
from pyomo.environ import *

# Initialize problem data
model = AbstractModel()
model.c = Param()
model.r = Param()
model.scens = Set()
model.d = Param(model.scens)

# Setup all stage problems
model.x = Var(within=NonNegativeReals)
model.y = Var(within=NonNegativeReals)

model.FirstStageCost = Var()
model.SecondStageCost = Var()
def obj_rule(model):
    return -model.c*model.x + model.r*model.y
model.obj = Objective(rule=obj_rule, sense=maximize)


model.constr1 = Constraint(expr=model.y <= model.x)

def cons2(model, s):
    return model.y <= model.d[s]
model.constr2 = Constraint(model.scens, rule=cons2)

##### Create a scenario data file named "ScenarioStructure.dat"

In [ ]:
"""
set Nodes := RootNode
             BelowAverageNode
             AverageNode
             AboveAverageNode;

param NodeStage := RootNode FirstStage
                   BelowAverageNode SecondStage
                   AverageNode SecondStage
                   AboveAverageNode SecondStage;

set Children[RootNode] := BelowAverageNode
                          AverageNode
                          AboveAverageNode;

param ConditionalProbability := RootNode 1.0
                                BelowAverageNode 0.333
                                AverageNode 0.333
                                AboveAverageNode 0.333;

set Scenarios := BelowAverageScenario
                 AverageScenario
                 AboveAverageScenario;

param ScenarioLeafNode :=
        BelowAverageScenario BelowAverageNode
        AverageScenario AverageNode
        AboveAverageScenario AboveAverageNode;

set StageVariables[FirstStage] := x;
set StageVariables[SecondStage] := y;

param c := 0.6;
param r := 1.5;
set scens := BELOW AVG ABOVE;
param d := BELOW 15 AVG 45 ABOVE 90;
"""

Run the following command through the terminal 

runef -m models -i nodedata --solver=cbc --solve